# Title

* **Difficulty level**: easy
* **Time need to lean**: 10 minutes or less
* **Key points**:
  * a
  

## `dryrun` mode

The `dryrun` mode is used to check for errors of 
SoS workflows without actually executing any of the actions and tasks. It can be specified with option `-n` to command `run` (`sos run -n`, `sos-runner -n`) or with subcommand `dryrun` (`sos dryrun`).

### Checking syntax error

The first use of dryrun mode is to check syntax error of scripts. For example, the following script yields an error because the header of the step is wrong (should use `:` instead of `,`).

In [1]:
%sandbox --expect-error
%run -n
[10, skip=False]
sh:
   echo "I am command echo"

File contains parsing errors: 
	[line  2]: [10, skip=False]
sh:
   echo "I am command echo"
Invalid statements: SyntaxError('invalid syntax', ('<string>', 1, 10, '[10, skip=False]\n'))


### Print instead of execute scripts

Script-executing actions such as `run`, `sh`, and `python` does not execute the scripts in dryrun mode. In stead, they will print the script with the command to execute them. This usage allows you to check if the scripts are interpolated correctly, or even executing them outside of SoS.

For example, the following script will print the expanded version of the scripts to be executed:

In [1]:
!rm -f test*.txt
%run -n

[10]
input: for_each={'i': (1, 2)}
output: f'test_{i}.txt'
sh: expand=True
  sleep {100*i}
  echo "Generating {_output}"
  touch {_output}
                 

/bin/sh SCRIPT

sleep 100
echo "Generating test_1.txt"
touch test_1.txt



/bin/sh SCRIPT

sleep 200
echo "Generating test_2.txt"
touch test_2.txt



The output files does not exist after the execution of the workflow.

In [2]:
!ls test*.txt

ls: test*.txt: No such file or directory


It is worth noting, however, that sos executes Python statements in steps as usual so your workflow will appear to be executed normally if you do not use actions.

For example, if you implement your steps in Python and include them directly, the step will be executed

In [3]:
!rm -f test*.txt
%run -n

[10]
input: for_each={'i': (1, 2)}
output: f'test_{i}.txt'
print("Generating {_output}")
_output.touch()

Generating {_output}
Generating {_output}


and the output files will be generated:

In [4]:
!ls test*.txt

test_1.txt test_2.txt


### Generation of placeholder files

Another consequence of running workflows in dryrun mode is that SoS generates empty placeholder files during dryrun and remove them afterwards. This allows the workflows to be executed "normally" because the execution of the next steps might depend on the existence of the output files.

For example, when running in dryrun mode, the following script will be executed "normally". The `_input` files (which are the output of step 10) could be opened at step 20, but the content would be empty:

In [5]:
!rm -f test*.txt
%run -n

[10]
input: for_each={'i': (1, 2)}
output: f'test_{i}.txt'
sh: expand=True
  echo "Generating {_output}" > {_output}
  
[20]
input: group_by=1
output: f'{_input}.bak'
print(f'Content of {_input}:')
with open(_input) as ifile:
    print(ifile.read())
sh: expand=True
  cp {_input} {_output}

/bin/sh SCRIPT

echo "Generating test_1.txt" > test_1.txt




/bin/sh SCRIPT

echo "Generating test_2.txt" > test_2.txt


Content of test_1.txt:



/bin/sh SCRIPT

cp test_1.txt test_1.txt.bak

Content of test_2.txt:



/bin/sh SCRIPT

cp test_2.txt test_2.txt.bak



`test_1.txt` and `test_2.txt` would be generated normally in run mode:

In [6]:
%rerun

Content of test_1.txt:
Generating test_1.txt

Content of test_2.txt:
Generating test_2.txt



The placeholder files are removed automatically after the execution of the workflow in dryrun mode. However, if you interrupt a script running in dryrun mode, some placeholder files might remain and interfere with the execution of workflows. In this case, you can use command

In [7]:
!sos remove --placeholders

INFO: No remaining placeholder file exists.


to remove such files. A shorter form of this command is `sos remove -p`.

### External tasks

External tasks are executed in `dryrun` mode as follows:

1. Only task from the first substep will be generated and submitted.
2. The task will be submitted to remote hosts as usual, with input files synchronized to remote host if needed. This allows you to test if the remote hosts are accessible.
3. The task will be executed in `dryrun` mode, and will print instead of execute the scripts defined in actions.

For example, although step `10` of the following workflow has 100 substeps, only the first task will be generated and executed in dryrun mode.

In [8]:
%run -n

[10]
input: for_each={'i': range(100)}
output: f'test_{i}.txt'
task:
sh: expand=True
  echo "Generating {_output}" > {_output}

1 task completed: a415

### Docker actions

When an action is executed in docker in dryrun mode, SoS would download the docker image if needed but will not execute the script. Instead, SoS will print the interpolated script and the docker command needs to execute the script. 

For example, running an `docker_build` action in dryrun mode yields the following output:

In [9]:
%run -n
[0]
docker_build:  tag='test/docker_build1', label='label with space', compress=True, memory='2G'
#
# Super simple example of a Dockerfile
#
FROM ubuntu:latest
MAINTAINER Andrew Odewahn "odewahn@oreilly.com"

WORKDIR /home

docker build /Users/bpeng1/sos/sos-docs/src/tutorials/tmp38mq9pwr --tag test/docker_build1 --label "label with space" --compress --memory 2G


#
# Super simple example of a Dockerfile
#
FROM ubuntu:latest
MAINTAINER Andrew Odewahn "odewahn@oreilly.com"

WORKDIR /home



and running a script in docker in dryrun mode:

In [10]:
%run -n
[0]
run: docker_image='ubuntu'
echo 'Echo'

docker run --rm -v /Users/bpeng1/sos/sos-docs/src/tutorials:/Users/bpeng1/sos/sos-docs/src/tutorials -v /Users/bpeng1/sos/sos-docs/src/tutorials/tmpu0aa6q67/docker_run_64107.sh:/var/lib/sos/docker_run_64107.sh -t -P -w=/Users/bpeng1/sos/sos-docs/src/tutorials -u 1985961928:895809667 ubuntu /bin/bash -ev /var/lib/sos/docker_run_64107.sh


echo 'Echo'



## Further reading

* 